# EGO Vehicle Trajectory Clustering

Extract features from the last 6 seconds of EGO vehicle trajectories from the first 100 train files and perform clustering analysis

In [ ]:
from pathlib import Path
import sys

project_root = Path.cwd().resolve()
if not (project_root / "src").exists():
    for parent in project_root.parents:
        if (parent / "src").exists():
            project_root = parent
            break

if (project_root / "src").exists() and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import torch
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from src.utils.data_visualization import get_available_files, DataLoader

sns.set_style('whitegrid')
print("✅ Libraries loaded")

## 1. Load data and extract EGO features

In [ ]:
# Configuration
DATA_PATH = Path('/data1/xiaowei/code/DeMo/data/DeMo_processed')
N_FILES = 1000 # 199908 for all training files
TOTAL_TIMESTEPS = 110  # 11 seconds * 10 Hz
HISTORY_TIMESTEPS = 50  # First 5 seconds
FUTURE_TIMESTEPS = 60   # Last 6 seconds

# Get files
train_files = get_available_files(DATA_PATH, 'train')[:N_FILES]
print(f"📂 Loaded {len(train_files)} files")

In [ ]:
def normalize_trajectory(positions, velocity, angles):
    """
    Normalize trajectory: start at origin, end aligned to y-axis
    
    Args:
        positions: numpy array of shape (60, 2) - [x, y] positions
        velocity: numpy array of shape (60,) - velocity values
        angles: numpy array of shape (60,) - heading angles
    
    Returns:
        features: numpy array of shape (60, 4) - [normalized_x, normalized_y, velocity, angle]
    """
    # 1. Translation: Move start to origin
    start_pos = positions[0]
    end_pos = positions[-1]
    positions_normalized = positions - start_pos  # (60, 2)
    
    # 2. Rotation alignment: Align start->end vector to positive y-axis direction
    # Calculate vector from start to end
    start_to_end = end_pos - start_pos
    target_angle = np.arctan2(start_to_end[1], start_to_end[0])  # Current angle
    
    # Calculate rotation angle (rotate vector to positive y-axis, i.e., 90 degrees)
    rotation_angle = np.pi / 2 - target_angle
    
    # Construct rotation matrix
    cos_theta = np.cos(rotation_angle)
    sin_theta = np.sin(rotation_angle)
    rotation_matrix = np.array([
        [cos_theta, -sin_theta],
        [sin_theta, cos_theta]
    ])
    
    # Apply rotation to positions
    positions_aligned = positions_normalized @ rotation_matrix.T  # (60, 2)
    
    # Apply rotation to angles (adjust all angles by rotation_angle)
    angles_aligned = angles - rotation_angle
    
    # 3. Feature combination: [normalized x, y, velocity, angle]
    features = np.column_stack([
        positions_aligned[:, 0],  # aligned x (lateral offset)
        positions_aligned[:, 1],  # aligned y (longitudinal progress)
        velocity,                 # original velocity
        angles_aligned            # aligned heading angles
    ])  # (60, 4)
    
    return features


def extract_ego_features(file_path):
    """
    Extract features from the last 6 seconds of EGO vehicle trajectory using DataLoader
    
    Args:
        file_path: Path to the data file
    
    Returns:
        tuple: (features, agent_type_combined) where:
            - features: numpy array of shape (60, 4) or None if invalid
            - agent_type_combined: string indicating agent type ('Vehicle', 'Pedestrian', 'Cyclist', 'Other')
    """
    # Use DataLoader to load and extract ego data
    loader = DataLoader()
    data = loader.load_scenario(file_path)
    
    if data is None:
        return None, None
    
    # Get focal agent index
    focal_idx = loader.current_metadata['focal_agent_idx']
    
    # Use plot_ego_velocity_analysis to get comprehensive ego data
    # This includes positions, velocities, angles, and accelerations
    from src.utils.data_visualization import plot_ego_velocity_analysis
    
    analysis_data = plot_ego_velocity_analysis(loader, show_acceleration=False, time_window=None)
    
    if analysis_data is None:
        return None, None
    
    # Extract agent_type_combined
    agent_type_combined = analysis_data['agent_type_combined']
    
    # Get full trajectory data
    positions_full = analysis_data['positions']
    velocities_full = analysis_data['velocities']
    angles_full = analysis_data['angles']
    timesteps_full = analysis_data['timesteps']
    
    # Filter for future timesteps (timestep 50-110, i.e., last 6 seconds)
    future_mask = timesteps_full >= HISTORY_TIMESTEPS
    
    if not future_mask.any():
        return None, None
    
    positions = positions_full[future_mask]
    velocity = velocities_full[future_mask]
    angles = angles_full[future_mask]
    
    # Check if we have exactly 60 timesteps
    if len(positions) != FUTURE_TIMESTEPS:
        return None, None
    
    # Normalize trajectory with positions, velocity, and angles from DataLoader
    features = normalize_trajectory(positions, velocity, angles)
    
    return features, agent_type_combined


# Extract all features and agent types
all_features = []
all_agent_types = []
valid_files = []

print("Extracting features using DataLoader with plot_ego_velocity_analysis...")
print("💡 This provides comprehensive data: positions, velocities, angles from the data loader")
for i, file_path in enumerate(train_files):
    if (i + 1) % 100 == 0:
        print(f"  Processing {i + 1}/{len(train_files)}...")
    
    features, agent_type = extract_ego_features(file_path)
    if features is not None:
        all_features.append(features)
        all_agent_types.append(agent_type)
        valid_files.append(file_path.name)

print(f"\n✅ Successfully extracted features from {len(all_features)} scenarios")
print(f"Feature dimensions: {all_features[0].shape}")
print(f"💡 Features include: [normalized_x, normalized_y, velocity, angle]")
print(f"💡 All data sourced from DataLoader's get_agent_trajectory method")

# Show agent type distribution
from collections import Counter
agent_type_counts = Counter(all_agent_types)
print(f"\n📊 Agent Type Distribution:")
for agent_type, count in sorted(agent_type_counts.items()):
    print(f"  {agent_type}: {count} ({count/len(all_agent_types)*100:.1f}%)")


## 2. Pure Heuristic-Based Classification into 5 Expert Categories

Classify all trajectories using heuristic rules into 5 expert categories:
- **Expert 1**: Lane Keeping (Including Straight Ahead/Lane Change)
- **Expert 2**: Turn Left (Including Turn from stop/Turn from moving state)
- **Expert 3**: Turn Right (Including Turn from stop/Turn from moving state)
- **Expert 4**: Constraint-Driven Deceleration (Including Stop/Yield/Junction/Congestion)
- **Expert 5**: Others (Long-tail and infrequent behaviors)

In [ ]:
def classify_trajectories_to_5_experts(features, agent_types):
    """
    Classify trajectories into 5 expert categories using heuristic rules:
    
    Expert 1: Lane Keeping (Including Straight Ahead/Lane Change)
    Expert 2: Turn Left (Including Turn from stop/Turn from moving state)
    Expert 3: Turn Right (Including Turn from stop/Turn from moving state)
    Expert 4: Constraint-Driven Deceleration (Including Stop/Yield/Junction/Congestion)
    Expert 5: Others (Long-tail and infrequent behaviors)
    
    Args:
        features: array of shape (n_samples, 60, 4) containing [x, y, velocity, angle]
        agent_types: list of agent_type_combined strings
    
    Returns:
        labels: array of shape (n_samples,) with values 1-5 representing expert categories
        label_names: dict mapping label to expert name
    """
    n_samples = len(features)
    labels = np.full(n_samples, 5, dtype=int)  # Default: Expert 5 (Others)
    
    # ===== Thresholds for classification =====
    WINDOW_SIZE = 5  # Number of timesteps to average for initial/final speed
    
    # Expert 4: Constraint-Driven Deceleration thresholds
    DECEL_FINAL_SPEED_THRESHOLD = 3.0  # m/s, final speed low or near stop
    DECEL_SPEED_DECREASE_THRESHOLD = -2.5  # m/s, significant speed decrease
    
    # Expert 1: Lane Keeping thresholds
    LANE_KEEPING_SPEED_STD_THRESHOLD = 2.0  # m/s, relatively stable speed
    LANE_KEEPING_ANGLE_STD_THRESHOLD = 0.2  # rad, low heading change (~11 degrees)
    LANE_KEEPING_LATERAL_THRESHOLD = 3.0  # m, small lateral deviation
    LANE_KEEPING_MIN_SPEED = 3.0  # m/s, minimum average speed
    
    # Expert 2 & 3: Turn detection thresholds
    TURN_ANGLE_CHANGE_THRESHOLD = 0.35  # rad, significant heading change (~20 degrees)
    TURN_TOTAL_ANGLE_THRESHOLD = 0.5  # rad, total angle change (~28 degrees)
    
    for i, sample in enumerate(features):
        agent_type = agent_types[i]
        
        # Non-vehicle agents go to Expert 5 (Others)
        if agent_type in ['Pedestrian']: # ['Pedestrian', 'Cyclist']
            labels[i] = 5  # Expert 5: Others
            continue
        
        # Extract features for vehicles
        velocities = sample[:, 2]
        angles = sample[:, 3]
        x_positions = sample[:, 0]
        y_positions = sample[:, 1]
        
        # Calculate statistics
        initial_speed = np.mean(velocities[:WINDOW_SIZE])
        final_speed = np.mean(velocities[-WINDOW_SIZE:])
        speed_change = final_speed - initial_speed
        avg_speed = np.mean(velocities)
        speed_std = np.std(velocities)
        angle_std = np.std(angles)
        lateral_deviation = np.std(x_positions)
        
        # Calculate total angle change (accumulated heading change)
        angle_diffs = np.diff(angles)
        # Normalize angle differences to [-pi, pi]
        angle_diffs = np.arctan2(np.sin(angle_diffs), np.cos(angle_diffs))
        total_angle_change = np.sum(angle_diffs)
        abs_total_angle_change = np.abs(total_angle_change)
        
        # Calculate final direction (last position relative to start)
        final_x = x_positions[-1] - x_positions[0]
        final_y = y_positions[-1] - y_positions[0]
        
        # ===== Classification Rules (priority order) =====
        
        # Rule 1: Expert 4 - Constraint-Driven Deceleration
        # Significant deceleration with low final speed
        if (final_speed < DECEL_FINAL_SPEED_THRESHOLD and 
            speed_change < DECEL_SPEED_DECREASE_THRESHOLD):
            labels[i] = 4  # Expert 4: Constraint-Driven Deceleration
        
        # Rule 2: Expert 2 - Turn Left
        # Significant angle change + net left turn (positive angle change in normalized coordinates)
        elif (abs_total_angle_change > TURN_TOTAL_ANGLE_THRESHOLD and 
              total_angle_change > TURN_ANGLE_CHANGE_THRESHOLD):
            labels[i] = 2  # Expert 2: Turn Left
        
        # Rule 3: Expert 3 - Turn Right
        # Significant angle change + net right turn (negative angle change)
        elif (abs_total_angle_change > TURN_TOTAL_ANGLE_THRESHOLD and 
              total_angle_change < -TURN_ANGLE_CHANGE_THRESHOLD):
            labels[i] = 3  # Expert 3: Turn Right
        
        # Rule 4: Expert 1 - Lane Keeping
        # Stable speed, stable heading, small lateral deviation, reasonable speed
        elif (speed_std < LANE_KEEPING_SPEED_STD_THRESHOLD and 
              angle_std < LANE_KEEPING_ANGLE_STD_THRESHOLD and
              lateral_deviation < LANE_KEEPING_LATERAL_THRESHOLD and
              avg_speed > LANE_KEEPING_MIN_SPEED):
            labels[i] = 1  # Expert 1: Lane Keeping
        
        # Rule 5: Expert 5 - Others (default, already set)
        # Includes: complex maneuvers, lane changes with acceleration, unusual patterns
    
    # Define label names
    label_names = {
        1: "Expert 1: Lane Keeping",
        2: "Expert 2: Turn Left",
        3: "Expert 3: Turn Right",
        4: "Expert 4: Constraint-Driven Deceleration",
        5: "Expert 5: Others"
    }
    
    return labels, label_names


# Apply the 5-expert classification
expert_labels, expert_names = classify_trajectories_to_5_experts(all_features, all_agent_types)

# Count samples in each category
print("=" * 70)
print("🎯 Pure Heuristic-Based Classification into 5 Expert Categories:")
print("=" * 70)
for expert_id in range(1, 6):
    count = np.sum(expert_labels == expert_id)
    percentage = count / len(all_features) * 100
    print(f"  {expert_names[expert_id]:<45s}: {count:4d} ({percentage:5.1f}%)")
print("=" * 70)
print(f"\n✅ Total classified samples: {len(expert_labels)}")

## 3. Visualize 5-Expert Classification Results

In [ ]:
# 1. Expert distribution statistics
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Get expert counts
unique_labels = sorted(np.unique(expert_labels))
label_names_list = []
expert_counts_list = []

# Define colors for 5 experts
expert_colors = {
    1: '#4ECDC4',  # Teal for Lane Keeping
    2: '#FFB84D',  # Orange for Turn Left
    3: '#FF6B6B',  # Red for Turn Right
    4: '#9B59B6',  # Purple for Constraint-Driven Deceleration
    5: '#95A5A6'   # Gray for Others
}

for label in unique_labels:
    count = np.sum(expert_labels == label)
    expert_counts_list.append(count)
    label_names_list.append(expert_names[label])

# Plot expert distribution bar chart
colors_bar = [expert_colors[label] for label in unique_labels]
axes[0].bar(range(len(expert_counts_list)), expert_counts_list, color=colors_bar, alpha=0.7)
axes[0].set_xticks(range(len(expert_counts_list)))
axes[0].set_xticklabels([f'Expert {i}' for i in unique_labels], rotation=0, ha='center')
axes[0].set_xlabel('Expert Category', fontsize=12)
axes[0].set_ylabel('Sample Count', fontsize=12)
axes[0].set_title('Sample Distribution Across 5 Expert Categories', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Plot expert distribution pie chart
axes[1].pie(expert_counts_list, labels=[f'Expert {i}' for i in unique_labels],
            autopct='%1.1f%%', colors=colors_bar, startangle=90)
axes[1].set_title('Expert Sample Proportions', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n📊 Expert Distribution Details:")
for label, name, count in zip(unique_labels, label_names_list, expert_counts_list):
    print(f"  {name:<50s}: {count:4d} ({count/len(expert_labels)*100:5.1f}%)")

In [ ]:
# 2. Typical trajectory visualization for each expert
unique_labels = sorted(np.unique(expert_labels))
n_experts = len(unique_labels)

# Create layout for experts
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Define colors for 5 experts
expert_colors = {
    1: '#4ECDC4',  # Teal for Lane Keeping
    2: '#FFB84D',  # Orange for Turn Left
    3: '#FF6B6B',  # Red for Turn Right
    4: '#9B59B6',  # Purple for Constraint-Driven Deceleration
    5: '#95A5A6'   # Gray for Others
}

# Get expert counts
expert_counts = {label: np.sum(expert_labels == label) for label in unique_labels}

for idx, expert_id in enumerate(unique_labels):
    # Get all samples for this expert
    expert_mask = expert_labels == expert_id
    expert_samples = [all_features[i] for i in range(len(all_features)) if expert_mask[i]]
    
    # Plot all trajectories for this expert (relative to start position)
    ax = axes[idx]
    for sample in expert_samples:
        # Convert to relative positions
        x_rel = sample[:, 0] - sample[0, 0]
        y_rel = sample[:, 1] - sample[0, 1]
        ax.plot(x_rel, y_rel, alpha=0.3, linewidth=1, color=expert_colors[expert_id])
    
    # Calculate and plot average trajectory (relative)
    mean_sample = np.mean(expert_samples, axis=0)
    mean_x_rel = mean_sample[:, 0] - mean_sample[0, 0]
    mean_y_rel = mean_sample[:, 1] - mean_sample[0, 1]
    ax.plot(mean_x_rel, mean_y_rel, color='darkblue', linewidth=3, label='Average Trajectory')
    
    ax.scatter(0, 0, color='green', s=100, marker='o', label='Start Point', zorder=5)
    ax.scatter(mean_x_rel[-1], mean_y_rel[-1], color='red', s=100, marker='*', label='End Point', zorder=5)
    
    # Set background color
    ax.set_facecolor(expert_colors[expert_id] + '20')  # 20 is alpha in hex
    
    ax.set_xlabel('Relative X Position (m)', fontsize=10)
    ax.set_ylabel('Relative Y Position (m)', fontsize=10)
    ax.set_title(f'{expert_names[expert_id]}\n({expert_counts[expert_id]} samples)', 
                 fontsize=12, fontweight='bold')
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.axis('equal')

# Hide unused subplot
if n_experts < 6:
    axes[5].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Optional: Load and verify the saved CSV
loaded_df = pd.read_csv(csv_path_latest)
print(f"✅ Loaded CSV with {len(loaded_df)} rows")
print(f"\nColumn types:")
print(loaded_df.dtypes)
print(f"\nSummary statistics:")
print(loaded_df.describe())

# Verify expert distribution matches
print(f"\n✅ Verification: Expert distribution")
for expert_id in range(1, 6):
    count_original = np.sum(expert_labels == expert_id)
    count_csv = len(loaded_df[loaded_df['expert_id'] == expert_id])
    match_status = "✓" if count_original == count_csv else "✗"
    print(f"  {match_status} Expert {expert_id}: Original={count_original}, CSV={count_csv}")

In [ ]:
# 3. Speed pattern visualization for each expert
unique_labels = sorted(np.unique(expert_labels))
n_experts = len(unique_labels)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Define colors for 5 experts
expert_colors = {
    1: '#4ECDC4',  # Teal for Lane Keeping
    2: '#FFB84D',  # Orange for Turn Left
    3: '#FF6B6B',  # Red for Turn Right
    4: '#9B59B6',  # Purple for Constraint-Driven Deceleration
    5: '#95A5A6'   # Gray for Others
}

expert_counts = {label: np.sum(expert_labels == label) for label in unique_labels}

for idx, expert_id in enumerate(unique_labels):
    expert_mask = expert_labels == expert_id
    expert_samples = [all_features[i] for i in range(len(all_features)) if expert_mask[i]]
    
    ax = axes[idx]
    
    # Plot velocity for all samples
    for sample in expert_samples:
        vel = sample[:, 2]
        ax.plot(vel, alpha=0.2, linewidth=1, color=expert_colors[expert_id])
    
    # Plot average velocity
    mean_vel = np.mean([s[:, 2] for s in expert_samples], axis=0)
    ax.plot(mean_vel, color='darkblue', linewidth=2.5, label='Average Speed')
    
    ax.axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    
    # Set background color
    ax.set_facecolor(expert_colors[expert_id] + '20')
    
    ax.set_xlabel('Time Step', fontsize=10)
    ax.set_ylabel('Velocity (m/s)', fontsize=10)
    ax.set_title(f'{expert_names[expert_id]} - Speed Pattern', fontsize=12, fontweight='bold')
    ax.legend(loc='best', fontsize=8)
    ax.grid(True, alpha=0.3)

# Hide unused subplot
if n_experts < 6:
    axes[5].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Optional: Load and verify the saved CSV
loaded_df = pd.read_csv(csv_path_latest)
print(f"✅ Loaded CSV with {len(loaded_df)} rows")
print(f"\nColumn types:")
print(loaded_df.dtypes)
print(f"\nSummary statistics:")
print(loaded_df.describe())

# Verify expert distribution matches
print(f"\n✅ Verification: Expert distribution")
for expert_id in range(1, 6):
    count_original = np.sum(expert_labels == expert_id)
    count_csv = len(loaded_df[loaded_df['expert_id'] == expert_id])
    match_status = "✓" if count_original == count_csv else "✗"
    print(f"  {match_status} Expert {expert_id}: Original={count_original}, CSV={count_csv}")

In [ ]:
# 4. Expert feature statistics comparison
unique_labels = sorted(np.unique(expert_labels))
expert_stats = []

for expert_id in unique_labels:
    expert_mask = expert_labels == expert_id
    expert_samples = [all_features[i] for i in range(len(all_features)) if expert_mask[i]]
    
    # Calculate statistics
    all_samples_array = np.array(expert_samples)  # (n_samples, 60, 4)
    
    stats = {
        'expert': expert_names[expert_id],
        'label': expert_id,
        'count': len(expert_samples),
        'avg_x_displacement': np.mean([s[-1, 0] - s[0, 0] for s in expert_samples]),  # Final - initial X
        'avg_y_displacement': np.mean([s[-1, 1] - s[0, 1] for s in expert_samples]),  # Final - initial Y
        'avg_total_displacement': np.mean([np.sqrt((s[-1, 0] - s[0, 0])**2 + (s[-1, 1] - s[0, 1])**2) for s in expert_samples]),
        'avg_speed_change': np.mean([np.sum(np.diff(s[:, 2])) for s in expert_samples]),  # Total speed change
        'speed_volatility': np.mean([np.std(s[:, 2]) for s in expert_samples])
    }
    expert_stats.append(stats)

# Convert to DataFrame and display
import pandas as pd
stats_df = pd.DataFrame(expert_stats)
stats_df = stats_df.round(3)

print("\n📈 Statistical Feature Comparison Across 5 Expert Categories:\n")
print(stats_df[['expert', 'count', 'avg_x_displacement', 'avg_y_displacement', 
                'avg_total_displacement', 'avg_speed_change', 'speed_volatility']].to_string(index=False))

# Visualize statistical features
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Define colors for 5 experts
expert_colors = {
    1: '#4ECDC4',  # Teal for Lane Keeping
    2: '#FFB84D',  # Orange for Turn Left
    3: '#FF6B6B',  # Red for Turn Right
    4: '#9B59B6',  # Purple for Constraint-Driven Deceleration
    5: '#95A5A6'   # Gray for Others
}

bar_colors = [expert_colors[label] for label in stats_df['label']]

# Average X Displacement
axes[0, 0].bar(range(len(stats_df)), stats_df['avg_x_displacement'], color=bar_colors)
axes[0, 0].set_xticks(range(len(stats_df)))
axes[0, 0].set_xticklabels([f'Expert {i}' for i in stats_df['label']], rotation=0, ha='center', fontsize=9)
axes[0, 0].set_ylabel('Average X Displacement (m)')
axes[0, 0].set_title('Average X Displacement Comparison')
axes[0, 0].grid(axis='y', alpha=0.3)

# Average Y Displacement
axes[0, 1].bar(range(len(stats_df)), stats_df['avg_y_displacement'], color=bar_colors)
axes[0, 1].set_xticks(range(len(stats_df)))
axes[0, 1].set_xticklabels([f'Expert {i}' for i in stats_df['label']], rotation=0, ha='center', fontsize=9)
axes[0, 1].set_ylabel('Average Y Displacement (m)')
axes[0, 1].set_title('Average Y Displacement Comparison')
axes[0, 1].grid(axis='y', alpha=0.3)

# Total Displacement
axes[0, 2].bar(range(len(stats_df)), stats_df['avg_total_displacement'], color=bar_colors)
axes[0, 2].set_xticks(range(len(stats_df)))
axes[0, 2].set_xticklabels([f'Expert {i}' for i in stats_df['label']], rotation=0, ha='center', fontsize=9)
axes[0, 2].set_ylabel('Average Total Displacement (m)')
axes[0, 2].set_title('Total Displacement Comparison')
axes[0, 2].grid(axis='y', alpha=0.3)

# Speed Change
axes[0, 3].bar(range(len(stats_df)), stats_df['avg_speed_change'], color=bar_colors)
axes[0, 3].set_xticks(range(len(stats_df)))
axes[0, 3].set_xticklabels([f'Expert {i}' for i in stats_df['label']], rotation=0, ha='center', fontsize=9)
axes[0, 3].set_ylabel('Average Speed Change (m/s)')
axes[0, 3].set_title('Average Speed Change Comparison')
axes[0, 3].grid(axis='y', alpha=0.3)

# Speed Volatility
axes[1, 0].bar(range(len(stats_df)), stats_df['speed_volatility'], color=bar_colors)
axes[1, 0].set_xticks(range(len(stats_df)))
axes[1, 0].set_xticklabels([f'Expert {i}' for i in stats_df['label']], rotation=0, ha='center', fontsize=9)
axes[1, 0].set_ylabel('Speed Volatility (m/s)')
axes[1, 0].set_title('Speed Volatility Comparison')
axes[1, 0].grid(axis='y', alpha=0.3)

# Average Speed
avg_speed = [np.mean([s[:, 2] for s in expert_samples]) for expert_samples in 
             [[all_features[i] for i in range(len(all_features)) if expert_labels[i] == label] 
              for label in unique_labels]]
axes[1, 1].bar(range(len(stats_df)), avg_speed, color=bar_colors)
axes[1, 1].set_xticks(range(len(stats_df)))
axes[1, 1].set_xticklabels([f'Expert {i}' for i in stats_df['label']], rotation=0, ha='center', fontsize=9)
axes[1, 1].set_ylabel('Average Speed (m/s)')
axes[1, 1].set_title('Average Speed Comparison')
axes[1, 1].grid(axis='y', alpha=0.3)

# Max Speed
max_speed = [np.max([np.max(s[:, 2]) for s in expert_samples]) for expert_samples in 
             [[all_features[i] for i in range(len(all_features)) if expert_labels[i] == label] 
              for label in unique_labels]]
axes[1, 2].bar(range(len(stats_df)), max_speed, color=bar_colors)
axes[1, 2].set_xticks(range(len(stats_df)))
axes[1, 2].set_xticklabels([f'Expert {i}' for i in stats_df['label']], rotation=0, ha='center', fontsize=9)
axes[1, 2].set_ylabel('Max Speed (m/s)')
axes[1, 2].set_title('Max Speed Comparison')
axes[1, 2].grid(axis='y', alpha=0.3)

# Angle Volatility
angle_volatility = [np.mean([np.std(s[:, 3]) for s in expert_samples]) for expert_samples in 
                    [[all_features[i] for i in range(len(all_features)) if expert_labels[i] == label] 
                     for label in unique_labels]]
axes[1, 3].bar(range(len(stats_df)), angle_volatility, color=bar_colors)
axes[1, 3].set_xticks(range(len(stats_df)))
axes[1, 3].set_xticklabels([f'Expert {i}' for i in stats_df['label']], rotation=0, ha='center', fontsize=9)
axes[1, 3].set_ylabel('Angle Volatility (rad)')
axes[1, 3].set_title('Angle Volatility Comparison')
axes[1, 3].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 5. Use t-SNE for dimensionality reduction visualization
from sklearn.manifold import TSNE

# Flatten time series data for t-SNE
X_flat = np.array([feat.flatten() for feat in all_features])

# t-SNE reduce to 2D
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_flat)

# Visualization with different colors for all categories
unique_labels = sorted(np.unique(expert_labels))

plt.figure(figsize=(12, 9))

# Define colors for 5 experts
expert_colors = {
    1: '#4ECDC4',  # Teal for Lane Keeping
    2: '#FFB84D',  # Orange for Turn Left
    3: '#FF6B6B',  # Red for Turn Right
    4: '#9B59B6',  # Purple for Constraint-Driven Deceleration
    5: '#95A5A6'   # Gray for Others
}

# Create color map
color_map = [expert_colors[label] for label in expert_labels]

scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=color_map, 
                      s=100, alpha=0.7, edgecolors='black', linewidth=0.5)

# Add expert center annotations
for expert_id in unique_labels:
    expert_mask = expert_labels == expert_id
    expert_center = X_tsne[expert_mask].mean(axis=0)
    
    marker_color = expert_colors[expert_id]
    label_text = f'E{expert_id}'
    
    plt.scatter(expert_center[0], expert_center[1], 
                marker='X', s=500, c=marker_color, edgecolors='black', linewidth=2)
    plt.text(expert_center[0], expert_center[1], label_text, 
             fontsize=12, fontweight='bold', ha='center', va='center',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

# Create custom legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor=expert_colors[1], edgecolor='black', label='Expert 1: Lane Keeping'),
    Patch(facecolor=expert_colors[2], edgecolor='black', label='Expert 2: Turn Left'),
    Patch(facecolor=expert_colors[3], edgecolor='black', label='Expert 3: Turn Right'),
    Patch(facecolor=expert_colors[4], edgecolor='black', label='Expert 4: Deceleration'),
    Patch(facecolor=expert_colors[5], edgecolor='black', label='Expert 5: Others')
]
plt.legend(handles=legend_elements, loc='best', fontsize=11)

plt.xlabel('t-SNE Dimension 1', fontsize=12)
plt.ylabel('t-SNE Dimension 2', fontsize=12)
plt.title('Trajectory Feature t-SNE Visualization\n(Pure Heuristic-Based 5-Expert Classification)', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("✅ t-SNE visualization completed")

## Save to CSV

In [ ]:
import pandas as pd
from datetime import datetime

# Create DataFrame with classification results
classification_results = []
for i, (filename, expert_id) in enumerate(zip(valid_files, expert_labels)):
    classification_results.append({
        'file_index': i,
        'filename': filename,
        'expert_id': int(expert_id)
        # 'expert_name': expert_names[expert_id],
        # 'agent_type': all_agent_types[i]
    })

df_classifications = pd.DataFrame(classification_results)

# Save to CSV
output_dir = Path('/data1/xiaowei/code/DeMo/data/DeMo_classified')
output_dir.mkdir(parents=True, exist_ok=True)

# Generate timestamped filename
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_filename = f'heuristic_classifications_{timestamp}.csv'
csv_path = output_dir / csv_filename

# Also save a latest version (no timestamp)
csv_path_latest = output_dir / 'heuristic_classifications_latest.csv'

# Save both versions
df_classifications.to_csv(csv_path, index=False)
df_classifications.to_csv(csv_path_latest, index=False)

print(f"✅ Classification results saved to:")
print(f"   📄 {csv_path}")
print(f"   📄 {csv_path_latest}")
print(f"\n📊 Saved {len(df_classifications)} classified scenarios")
print(f"\nFirst 5 rows:")
print(df_classifications.head())
print(f"\nCSV columns: {list(df_classifications.columns)}")
print(f"\nExpert distribution in saved CSV:")
print(df_classifications['expert_id'].value_counts().sort_index())

In [ ]:
# Optional: Load and verify the saved CSV
loaded_df = pd.read_csv(csv_path_latest)
print(f"✅ Loaded CSV with {len(loaded_df)} rows")
print(f"\nColumn types:")
print(loaded_df.dtypes)
print(f"\nSummary statistics:")
print(loaded_df.describe())

# Verify expert distribution matches
print(f"\n✅ Verification: Expert distribution")
for expert_id in range(1, 6):
    count_original = np.sum(expert_labels == expert_id)
    count_csv = len(loaded_df[loaded_df['expert_id'] == expert_id])
    match_status = "✓" if count_original == count_csv else "✗"
    print(f"  {match_status} Expert {expert_id}: Original={count_original}, CSV={count_csv}")